#### Code Design and Conditions for Successful Execution

#1. Ensure that the input Excel file has four columns before the city-specific data begins. If there are only three columns, add a blank column at the start named 'Relative'.

#2. The 'max score' column should contain numerical values for the rows where scores are to be calculated. If a row that describes a parameter has a score in the first row(in Upkeep) but needs to be applied to a cumulative total in the last row, ensure the 'max score' data is moved to the last row (total upkeep).

#3.  Maintain the exact row arrangement and nomenclature as provided in the attached input file.

#4. The code is designed to accommodate changes in scoring conditions dynamically. Modify conditions or introduce new ones by adjusting the code under Part4.

#5. For any parameter or formula involving non-numerical data, refer to Part6 in the code. Adjust conditions or introduce new ones based on the specific requirements.

#### Code Explanation Flow Chart

1. Load Input Excel File
   - Read the Excel file specified by `input_file_path`.
   - Clean strings in all cells to remove unwanted characters and spaces.

2. Prepare Output DataFrame
   - Create an output DataFrame (`output_df`) with the same structure as the input, including four initial columns.
   - Add 'Maximum' and 'Ratio' columns to store calculation results.

3. Clean 'max score' Values
   - Convert 'max score' values to float to ensure numerical consistency for scoring.

4. Calculate Scores for Each Parameter and City
   - Iterate over each row in the input DataFrame.
   - Determine the maximum value from numerical city-specific data.
   - Calculate a ratio for scoring based on the 'max score' and the maximum value found.
   - Apply conditions defined in Part6 to calculate scores for each city under each parameter.

5. Sum and Rank Calculation
   - Calculate the sum of scores for each city across all parameters.
   - Determine ranks based on these sums using a ranking method that handles ties by assigning the lowest rank.

6. Append Sum and Rank to Output DataFrame
   - Add rows to the output DataFrame for 'Sum of Scores' and 'Rank', including values calculated in the previous step.

7. Save Output to Excel
   - Write the final output DataFrame to a new Excel file specified by `output_file_path`.

8. Display Confirmation
   - Print a message confirming the successful saving of the output file.


### Code

In [2]:
import pandas as pd

# Part 1: Loading of dataset

input_file_path = 'sample2.xlsx'
output_file_path = 'MV_Score_Sample2.xlsx'

# Part 2

def clean_string(s): # function to clean strings
    if isinstance(s, str):
        return ' '.join(s.replace('_x000D_', '').split()).strip()
    return s


input_df = pd.read_excel(input_file_path)


input_df = input_df.applymap(clean_string) # cleaning dataframe

# Prepare the output DataFrame with the same structure as input
output_df = input_df.iloc[:, :4].copy()

# Part 3

output_df['Maximum'] = None # maximum column
output_df['Ratio'] = None # ratio column


city_columns = input_df.columns[4:] # city column
for city in city_columns:
    output_df[city] = None


def clean_max_score(value): # clean 'max score' values
    try:
        return float(value)
    except ValueError:
        return float('nan')


input_df['max score'] = input_df['max score'].apply(clean_max_score) # Clean the 'max score' column

# Part 4
# Function to calculate the score based on various conditions
def calculate_score(val, parameter, max_score, ratio, previous_values):
    if isinstance(val, (int, float)):
        if 'No. of new green areas created' in parameter:
            score = min(val * 10, 100)
        elif 'Number of meetings conducted' in parameter:
            score = min(val * 5, 20)
        elif 'Total Upkeep:Number of old spots (created during MVA 1.0,2.0,3.0)' in parameter:
            score = min(val * 25, 250)
        elif 'Total Upkeep: Maintenance of total green areas created during Majhi Vasundhara Abhiyan' in parameter:
            score = min(val * 5, 100)
        elif 'Usage of compost' in parameter:
            if val > 90:
                score = 70
            elif val > 75:
                score = 60
            elif val > 50:
                score = 50
            else:
                score = 0
        elif 'Segregation at source' in parameter or 'Collection' in parameter:
            if val > 95:
                score = 25
            elif val > 80:
                score = 15
            else:
                score = 0
        elif '% of wet waste processed' in parameter:
            if val > 90:
                score = 50
            elif val > 75:
                score = 40
            elif val > 50:
                score = 30
            else:
                score = 0
        elif 'Secondary Segregation of dry waste collected' in parameter:
            score = 10 if val >= 90 else 0
        elif '% of dry waste processed' in parameter:
            if val > 80:
                score = 30
            elif val > 50:
                score = 20
            else:
                score = 0
        elif 'Number of reports' in parameter:
            if val > 100:
                score = 200
            elif val > 75:
                score = 150
            elif val > 50:
                score = 100
            else:
                score = 0
        elif 'Percentage of projects for which NOC was processed' in parameter or 'Percentage of Compensatory plantation done as per NOCs granted by the Local body' in parameter:
            score = 50 if val == 100 else 0
        elif 'Identification of land/area for C&D waste storage' in parameter or 'Dedicated vehicles for collection' in parameter:
            score = 20 if val > 0 else 0
        elif 'Mapping of all wells in the Local body with geotagging.' in parameter or 'Identification of dysfunctional wells.' in parameter:
            score = 20 if val > 0 else 0
        elif 'Rejuvenation/Recharge of dysfunctional wells.' in parameter:
            score = 40 if val > 0 else 0
        elif 'Air quality monitoring report from – MoEFCC recognized/NABL accredited labs' in parameter:
            if val >= 8:
                score = 100
            elif val >= 6:
                score = 50
            elif val >= 5:
                score = 25
            else:
                score = 0
        elif 'Air Quality Monitoring Report on the evening /Next morning of the festival - with AQI as per the National Air Quality Index' in parameter:
            if val == 0:
                score = 0
            elif val <= 100:
                score = 75
            elif val <= 200:
                score = 50
            else:
                score = 0
        elif 'Number of Air Quality Monitoring stations, without visible public display' in parameter:
            if val == 0:
                score = ratio
            else:
                score = 0
        elif 'Percentage of LED Streetlights of total Streetlights' in parameter:
            score = 0 if val > 100 else val
        elif '% of govt. buildings with water audit conducted during Majhi Vasundhara Abhiyan 3.0 and/or Majhi Vasundhara Abhiyan 4.0' in parameter:
            if val == 100:
                score = 50
            elif val >= 75:
                score = 40
            elif val >= 50:
                score = 25
            elif val >= 25:
                score = 15
            else:
                score = 0
        elif 'Percentage of Public Buildings with functioning Rainwater harvesting projects installed before and during Majhi Vasundhara Abhiyan 4.0' in parameter:
            if val == 100:
                score = 100
            elif val >= 75:
                score = 75
            elif val >= 50:
                score = 50
            else:
                score = 0
        elif 'Percentage of treated water from STP/FSTP reused' in parameter:
            if val >= 100:
                score = 50
            elif val >= 80:
                score = 40
            elif val >= 50:
                score = 30
            else:
                score = 0
        elif '% of public buildings with energy audit conducted during Majhi Vasundhara Abhiyan 3.0 and Majhi Vasundhara Abhiyan 4.0.' in parameter:
            if val == 100:
                score = 50
            elif val >= 75:
                score = 40
            elif val >= 50:
                score = 25
            else:
                score = 0
        elif 'Individually continuous top performers in all 3 quarters (Top 10 Ranks from every quarter will be considered' in parameter:
            score = 0
        elif 'In group continuous top performers in all 3 quarters (Top 10 Ranks from every quarter will be considered)' in parameter:
            score = 0
        elif 'Number of new spots Developed with focus on Majhi Vasundhara Principles during Majhi Vasundhara Abhiyan 4.0' in parameter:
            score = min(val * 100, 500)
        else:
            score = val * ratio
    elif isinstance(val, str):
        if val == 'Yes':
            score = max_score
        elif val == 'No':
            score = 0
        elif val == 'Water +':
            score = 50
        elif val == 'ODF ++':
            score = 40
        elif val == 'ODF+' or 'ODF +':
            score = 20
        elif val == 'a. 100% of legacy waste treated and processed scientifically':
            score = 100
        elif val == '75% to less than 100% of legacy waste is treated and processed':
            score = 75
        elif val == '25% to less than 75% of legacy waste is treated and processed':
            score = 50
        elif val == 'Less than 25% of legacy waste is treated and processed':
            score = 0
        elif val == '7 star':
            score = 50
        elif val == '5 star':
            score = 35
        elif val == '3 star':
            score = 25
        elif val == '1 star':
            score = 10
        elif val == 'ULB Re-certified with HARIT':
            score = 30
        elif val == 'ULB certified with HARIT Brand':
            score = 20
        elif val == 'achieved 33% green cover and have a Tree plan':
            score = 50
        elif val == 'not achieved 33% green cover and has a Tree Plan':
            score = 50
        elif val == 'achieved 33% green cover but they have not prepared plan':
            score = 0
        elif val == 'not achieved 33% green cover and they have not':
            score = 0
        elif 'Tree Census with geotagging –100% report prepared and published' in val:
            row_idx = input_df[input_df['Parameter'] == 'Tree Census with Geotagging – Report Prepared and Published before Majhi Vasundhara Abhiyan 4.0'].index[0]
            score = 75 if input_df.loc[row_idx, city] == 'No' else 0
        elif 'Tree Census with geotagging –50% report prepared and published' in val:
            row_idx = input_df[input_df['Parameter'] == 'Tree Census with Geotagging – Report Prepared and Published before Majhi Vasundhara Abhiyan 4.0'].index[0]
            score = 35 if input_df.loc[row_idx, city] == 'No' else 0
        elif val == '95% or more':
            score = 50
        elif val == '75% - 95%':
            score = 25
        elif val == 'Less than 75%':
            score = 0
        else:
            score = None
    else:
        score = None
    return score

# Part 5
# Iterate over the rows and apply calculations
for idx, row in input_df.iterrows():
    if pd.notnull(row['max score']):
        city_data = row[4:]
        city_data_numeric = city_data.apply(pd.to_numeric, errors='coerce')  # Convert to numeric, set non-numeric to NaN
        max_value = city_data_numeric.max(skipna=True)  # Find max ignoring NaNs
        ratio = row['max score'] / max_value if max_value != 0 else 0
        scores = []
        for city, val in city_data.items():
            previous_values = input_df.iloc[max(0, idx-4):idx, 4:].values.flatten().tolist()
            score = calculate_score(val, row['Parameter'], row['max score'], ratio, previous_values)
            scores.append(score)
        output_df.at[idx, 'Maximum'] = max_value
        output_df.at[idx, 'Ratio'] = ratio
        output_df.iloc[idx, 6:] = scores


city_sums = output_df.iloc[:, 6:].sum()  # Calculate sum of scores for each city

# Determine ranks based on sums
city_ranks = city_sums.rank(method='min', ascending=False).astype(int)

# Append the sums and ranks to the output DataFrame using pd.concat
sum_row = pd.DataFrame({'Parameter': ['Sum of Scores'], 'Maximum': [None], 'Ratio': [None], **{city: [city_sums[city]] for city in city_columns}})
rank_row = pd.DataFrame({'Parameter': ['Rank'], 'Maximum': [None], 'Ratio': [None], **{city: [city_ranks[city]] for city in city_columns}})
output_df = pd.concat([output_df, sum_row, rank_row], ignore_index=True)


output_df.to_excel(output_file_path, index=False)

print(f"Output file saved at: {output_file_path}")


C:\Users\Atique\AppData\Local\Temp\ipykernel_8196\310021597.py:19: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  input_df = input_df.applymap(clean_string)


Output file saved at: MV_Score_Sample2.xlsx


#### @copyright